---
# Imports

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

---
# Data Import

In [ ]:
data = pd.read_csv('data/data_last.csv')
data = data[data['Type'] == 'pm10']

In [ ]:
data.info()

---
# Data Preparation

### Time stamp conversion

In [ ]:
data['Stamp'] = pd.to_datetime(data['Stamp'], errors='coerce', utc=True)

In [ ]:
data.info()

### Time Based Features

In [ ]:
data['Hour'] = data['Stamp'].dt.hour + data['Stamp'].dt.minute / 60

In [ ]:
data['Month_Day_Calculation'] = data['Stamp'].apply(
    lambda date: date.month + date.day / pd.Timestamp(f"{date.year}-{date.month:02d}-01").days_in_month
)

### Data Resampling and Interpolation by SensorId

In [ ]:
df = data.pivot_table(index=['SensorId', 'Stamp'], columns='Type', values='Value')

In [ ]:
df.set_index('Stamp', inplace=True)
df_resampled = df.groupby('SensorId').resample('15T').first()

In [ ]:
df[['humidity', 'pm10', 'pm25', 'temperature']] = df.groupby('SensorId')[['humidity', 'pm10', 'pm25', 'temperature']].transform(
    lambda group: group.interpolate(method='cubic', limit_direction='both')
)